In [ ]:
# %%

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
import os
import datetime
import time
from keras.models import model_from_json

# %%

wtp_in = 'bukkou_in'
wtp_out = 'sakuradai'

TS = [12, 30, 36, 42, 48]
F = [6]

# M = ['4_7', '5_8', '6_9', '7_10', '8_11', '9_12', '10_1', '11_2', '12_3']
# M = ['5', '7', '24', '25']
M = ['25']

for m in M:

    
    df1 = pd.read_csv('split_nishiya_bukkou/train' + m + '.csv')
    df1['target'] = df1[wtp_in] - df1[wtp_out]
    df1 = df1['target']
    df1 = pd.DataFrame(df1)
    df11 = df1['target']
    df11 = pd.DataFrame(df11)

    df2 = pd.read_csv('split_nishiya_bukkou/val' + m + '.csv')
    df2['target'] = df2[wtp_in] - df2[wtp_out]
    # df22 = df2["bukkou_out"] - df2[wtp]
    df2 = df2['target']
    df2 = pd.DataFrame(df2)
    df22 = df2['target']
    df22 = pd.DataFrame(df22)

    df3 = pd.read_csv('split_nishiya_bukkou/test' + m + '.csv')
    df3['target'] = df3[wtp_in] - df3[wtp_out]
    # df33 = df3["bukkou_out"] - df3[wtp]
    df3 = df3['target']
    df3 = pd.DataFrame(df3)
    df33 = df3['target']
    df33 = pd.DataFrame(df33)



    for ts in TS:
        for f in F:
            n_in = 1
            n_hidden = 1024
            n_out = 1

            epochs = 50
            batch_size = 64

            data = []
            target = []
            data1 = []
            target1 = []
            data2 = []
            target2 = []

            time_steps = ts
            future = f
            df_save = df1

            df1 = (df1 - df_save.min()) / (df_save.max() - df_save.min())

            for i in range(len(df1) - time_steps - future):
                data.append(df1.values[i: i + time_steps])

            for i in range(len(df11) - time_steps - future):
                target.append(df11.values[i + time_steps + future])

            x_train = np.array(data)
            y_train = np.array(target)

            x_train = x_train.reshape(len(x_train), time_steps, n_in)
            y_train = y_train.reshape(len(y_train), n_out)


            df2 = (df2 - df_save.min()) / (df_save.max() - df_save.min())

            for i in range(len(df2) - time_steps - future):
                data1.append(df2.values[i: i + time_steps])

            for i in range(len(df22) - time_steps - future):
                target1.append(df22.values[i + time_steps + future])

            x_val = np.array(data1)
            y_val = np.array(target1)

            x_val = x_val.reshape(len(x_val), time_steps, n_in)
            y_val = y_val.reshape(len(y_val), n_out)

            df3 = (df3 - df_save.min()) / (df_save.max() - df_save.min())

            for i in range(len(df3) - time_steps - future):
                data2.append(df3.values[i: i + time_steps])

            for i in range(len(df33) - time_steps - future):
                data2.append(df33.values[i: i + time_steps])

            x_test = np.array(data2)
            y_test = np.array(target2)

            x_test = x_test.reshape(len(x_test), time_steps, n_in)
            y_test = y_test.reshape(len(y_test), n_out)

            ES = EarlyStopping(monitor='val_loss', patience=5, verbose=1)


            model1 = Sequential()
            input_shape = (time_steps, n_in)
            model1.add(LSTM(n_hidden, input_shape=input_shape, activation='elu'))
            # model1.add(Dropout(0.5))
            model1.add(Dense(n_out))

            optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
            model1.compile(loss="mse", optimizer=optimizer)

            reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=3, factor=0.1, min_lr=1e-6,
                                            mode='auto', verbose=1)

            new_folder = wtp_in + '_' + wtp_out  + "_ts" + str(ts) + "_f" + str(f)
            if not os.path.exists(new_folder): 
                os.mkdir(new_folder) 


            model_checkpoint = ModelCheckpoint(
                filepath=os.path.join(new_folder, 'model.h5'),
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=True,
                verbose=1)

            history = model1.fit(x_train, y_train,
                                    batch_size=batch_size,
                                    epochs=epochs,
                                    verbose=2,
                                    validation_data=(x_val, y_val),
                                    callbacks=[ES, reduce_lr, model_checkpoint]
                                    )


            open(new_folder + '/' + 'model.json', "w").write(model1.to_json())
            model1.save_weights(new_folder + '/' + 'last_model.h5')

            json_file = open(new_folder + '/' + 'model.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            model = model_from_json(loaded_model_json)

            weightPath = new_folder + '/' + 'model.h5'
            if not os.path.exists(new_folder):
                model.load_weights(new_folder + '/' + 'last_model.h5')
            else:
                model.load_weights(new_folder + '/' + 'model.h5')


            optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
            model.compile(loss="mse", optimizer=optimizer)


            train_predict = model1.predict(x_train)

            train_predict = pd.DataFrame(train_predict)
            train_predict.to_csv(new_folder + '/' + new_folder + '_train_predict.csv')

            y_train = pd.DataFrame(y_train)
            y_train.to_csv(new_folder + '/' + new_folder + '_y_train.csv')

            val_predict = model1.predict(x_val)

            val_predict = pd.DataFrame(val_predict)
            val_predict.to_csv(new_folder + '/' + new_folder + '_val_predict.csv')

            y_val = pd.DataFrame(y_val)
            y_val.to_csv(new_folder + '/' + new_folder + '_y_val.csv')

            test_predict = model1.predict(x_test)

            test_predict = pd.DataFrame(test_predict)
            test_predict.to_csv(new_folder + '/' + new_folder + '_test_predict.csv')

            y_test = pd.DataFrame(y_test)
            y_test.to_csv(new_folder + '/' + new_folder + '_y_test.csv')

            plt.plot(y_test, label='test', color='black', linewidth=0.7)
            plt.plot(test_predict, label='pred', color='blue', linewidth=0.5)
            plt.legend(loc='upper right')
            plt.title("test")
            plt.savefig(new_folder + '/' + new_folder + '_LSTM_test.pdf')
            plt.show()

            loss = history.history['loss']
            val_loss = history.history['val_loss']
            epochs = len(loss)
            plt.plot(range(epochs), loss, marker='.', label='loss')
            plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
            plt.legend(loc='best')
            plt.grid()
            plt.xlabel('epoch')
            plt.ylabel('loss')
            plt.savefig(new_folder + '/' + new_folder + '_LSTM_epochs.pdf')
            plt.show()

